Dado que el entrenamiento de redes neuronales es una tarea  muy costosa, **se recomienda ejecutar el notebooks en [Google Colab](https://colab.research.google.com)**, por supuesto también se puede ejecutar en local.

Al entrar en [Google Colab](https://colab.research.google.com) bastará con hacer click en `upload` y subir este notebook. No olvide luego descargarlo en `File->Download .ipynb`

**El examen deberá ser entregado con las celdas ejecutadas, si alguna celda no está ejecutadas no se contará.**

El examen se divide en tres partes, con la puntuación que se indica a continuación. La puntuación máxima será 10.

    
- [Actividad 1: Redes Recurrentes](#actividad_1): 10 pts
    - [Cuestión 1](#3.1): 2.5 pt
    - [Cuestión 2](#3.2): 2.5 pt
    - [Cuestión 3](#3.3): 2.5 pts
    - [Cuestión 4](#3.4): 1.25 pts
    - [Cuestión 5](#3.5): 1.25 pts



In [92]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import random

<a name='actividad_1'></a>
# Actividad 1: Redes Recurrentes


- [Cuestión 1](#3.1): 2.5 pt
- [Cuestión 2](#3.2): 2.5 pt
- [Cuestión 3](#3.3): 2.5 pts
- [Cuestión 4](#3.4): 1.25 pts
- [Cuestión 5](#3.5): 1.25 pts

Vamos a usar un dataset de las temperaturas mínimas diarias en Melbourne. La tarea será la de predecir la temperatura mínima en dos días. Puedes usar técnicas de series temporales vistas en otras asignaturas, pero no es necesario.


In [93]:
dataset_url = 'https://raw.githubusercontent.com/jbrownlee/Datasets/master/daily-min-temperatures.csv'
data_dir = tf.keras.utils.get_file('daily-min-temperatures.csv', origin=dataset_url)

In [94]:
df = pd.read_csv(data_dir, parse_dates=['Date'])
df.head()

,Date,Temp
0,1981-01-01,20.7
1,1981-01-02,17.9
2,1981-01-03,18.8
3,1981-01-04,14.6
4,1981-01-05,15.8


In [95]:
temperatures = df['Temp'].values
print('number of samples:', len(temperatures))
train_data = temperatures[:3000]
test_data = temperatures[3000:]
print('number of train samples:', len(train_data))
print('number of test samples:', len(test_data))
print('firsts trainn samples:', train_data[:10])

number of samples: 3650
number of train samples: 3000
number of test samples: 650
firsts trainn samples: [20.7 17.9 18.8 14.6 15.8 15.8 15.8 17.4 21.8 20. ]


<a name='3.1'></a>
## Cuestión 1: Convierta `train_data` y `test_data`  en ventanas de tamaño 5, para predecir el valor en 2 días

En la nomenclatura de [Introduction_to_RNN_Time_Series.ipynb](https://github.com/ezponda/intro_deep_learning/blob/main/class/RNN/Introduction_to_RNN_Time_Series.ipynb)
```python
past, future = (5, 2)
```

Para las primeras 10 muestras de train_data `[20.7, 17.9, 18.8, 14.6, 15.8, 15.8, 15.8, 17.4, 21.8, 20. ]` el resultado debería ser:

```python
x[0] : [20.7, 17.9, 18.8, 14.6, 15.8] , y[0]: 15.8
x[1] : [17.9, 18.8, 14.6, 15.8, 15.8] , y[1]: 17.4
x[2] : [18.8, 14.6, 15.8, 15.8, 15.8] , y[2]: 21.8
x[3] : [14.6, 15.8, 15.8, 15.8, 17.4] , y[3]: 20.             
```

In [96]:
seed_value = 42
np.random.seed(seed_value)
tf.random.set_seed(seed_value)
random.seed(seed_value)

In [97]:
# windowing function
def create_windows_np(data, window_size, horizon, shuffle=False):
    """
    Creates a dataset from the given time series data using NumPy.
    
    Parameters:
    data (np.ndarray): Time series data with one dimension.
    window_size (int): The number of past time steps to use as input features.
    horizon (int): The number of future time steps to predict.
    shuffle (bool): Shuffle the windows or not.
    
    Returns:
    tuple: A tuple containing the input-output pairs (windows, targets) as NumPy arrays.
    """

    X, y = [], []
    for i in range(len(data) - window_size - horizon + 1):
        X.append(data[i:i+window_size])
        y.append(data[i+window_size+horizon-1])

    X, y = np.array(X), np.array(y)
    
    if shuffle:
        indices = np.arange(len(X))
        np.random.shuffle(indices)
        X, y = X[indices], y[indices]
    
    return X, y

In [98]:
past, future = (5, 2)
X_train, y_train = create_windows_np(train_data, past, future)
X_test, y_test = create_windows_np(test_data, past, future)

In [99]:
for i in range(10):
    print(f"x[{i}]: {X_train[i]},  y[{i}]: {y_train[i]}")

x[0]: [20.7 17.9 18.8 14.6 15.8],  y[0]: 15.8
x[1]: [17.9 18.8 14.6 15.8 15.8],  y[1]: 17.4
x[2]: [18.8 14.6 15.8 15.8 15.8],  y[2]: 21.8
x[3]: [14.6 15.8 15.8 15.8 17.4],  y[3]: 20.0
x[4]: [15.8 15.8 15.8 17.4 21.8],  y[4]: 16.2
x[5]: [15.8 15.8 17.4 21.8 20. ],  y[5]: 13.3
x[6]: [15.8 17.4 21.8 20.  16.2],  y[6]: 16.7
x[7]: [17.4 21.8 20.  16.2 13.3],  y[7]: 21.5
x[8]: [21.8 20.  16.2 13.3 16.7],  y[8]: 25.0
x[9]: [20.  16.2 13.3 16.7 21.5],  y[9]: 20.7


<a name='3.2'></a>
## Cuestión 2: Cree un modelo recurrente de dos capas GRU para predecir con las ventanas de la cuestión anterior.


In [100]:
inputs = keras.layers.Input(shape=(5, 1))
# Añadir las capas GRU. Dado que queremos apilar GRU, necesitamos que la primera devuelva secuencias
x = keras.layers.GRU(64, return_sequences=True)(inputs)
x = keras.layers.GRU(64)(x)

# Añadir una capa densa para la salida, queremos predecir un solo valor
outputs = keras.layers.Dense(1)(x)

# Crear y compilar el modelo
model = keras.Model(inputs=inputs, outputs=outputs)
model.compile(optimizer='adam', loss='mse')
model.summary()

Model: "functional_25"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_12 (InputLayer)     │ (None, 5, 1)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ gru_8 (GRU)                     │ (None, 5, 64)          │        12,864 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ gru_9 (GRU)                     │ (None, 64)             │        24,960 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_27 (Dense)                │ (None, 1)              │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 37,889 (148.00 KB)

 Trainable params: 37,889 (148.00 KB)

 Non-trainable params: 0 (0.00 B)

In [101]:
es_callback = keras.callbacks.EarlyStopping(
    monitor="val_loss", min_delta=0, patience=10)

history = model.fit(
    X_train, y_train,
    epochs=200,
    validation_split=0.2, shuffle=True, batch_size = 64, callbacks=[es_callback]
)

Epoch 1/200


38/38 ━━━━━━━━━━━━━━━━━━━━ 5s 23ms/step - loss: 73.0039 - val_loss: 20.8326
Epoch 2/200
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - loss: 16.4951 - val_loss: 18.1716
Epoch 3/200
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 14.9107 - val_loss: 15.6878
Epoch 4/200
38/38 ━━━━━━━━━━━━━━━━━━━━ 1s 16ms/step - loss: 12.3114 - val_loss: 12.2268
Epoch 5/200
38/38 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step - loss: 10.5327 - val_loss: 10.2985
Epoch 6/200
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - loss: 9.6520 - val_loss: 9.5350
Epoch 7/200
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - loss: 9.2589 - val_loss: 9.1977
Epoch 8/200
38/38 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - loss: 9.1023 - val_loss: 9.0360
Epoch 9/200
38/38 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step - loss: 9.0315 - val_loss: 8.9282
Epoch 10/200
38/38 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - loss: 8.9845 - val_loss: 8.8427
Epoch 11/200
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - loss: 8.9492 - val_loss: 8.7756
Epoch 12/200
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step -

In [102]:
results = model.evaluate(X_test, y_test, verbose=1)
print('Test Loss: {}'.format(results))

21/21 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 6.8167
Test Loss: 6.737891674041748


<a name='3.3'></a>
## Cuestión 3: Añada más features a la series temporal, por ejemplo `portion_year`. Cree un modelo que mejore al anterior.


In [103]:
## Puede añadir más features
df['portion_year'] = df['Date'].dt.dayofyear / 365.0
df['day_of_week'] = df['Date'].dt.dayofweek / 6.0  # Normalized
df['month_of_year'] = df['Date'].dt.month / 12.0  # Normalized

df_multi = df[['Temp', 'portion_year', 'day_of_week', 'month_of_year']].copy()

## train - test split
train_data = df_multi.iloc[:3000].copy()
test_data = df_multi.loc[3000:, :].copy()


In [104]:
def create_windows_multivariate_np(data, window_size, horizon, target_col_idx, shuffle=False):
    """
    Creates a dataset from the given time series data using NumPy.

    Parameters:
    data (np.ndarray or pd.DataFrame): Time series data with multiple features.
    window_size (int): The number of past time steps to use as input features.
    horizon (int): The number of future time steps to predict.
    target_col_idx (int): The index of the target column in the input data.
    shuffle (bool): Whether to shuffle the data or not.

    Returns:
    tuple: A tuple containing the input-output pairs (X, y) as NumPy arrays.
    """
    if isinstance(data, pd.DataFrame):
        data = data.values

    X, y = [], []
    for i in range(len(data) - window_size - horizon + 1):
        X.append(data[i:i+window_size, :])
        y.append(data[i+window_size+horizon-1, target_col_idx])

    X, y = np.array(X), np.array(y)

    if shuffle:
        indices = np.arange(X.shape[0])
        np.random.shuffle(indices)
        X, y = X[indices], y[indices]

    return X, y

In [105]:
## Create windows
X_train, y_train = create_windows_multivariate_np(train_data, past, future, 0)
X_test, y_test = create_windows_multivariate_np(test_data, past, future, 0)

for i in range(10):
    print(f"x[{i}]: {X_train[i]},  y[{i}]: {y_train[i]}")

x[0]: [[2.07000000e+01 2.73972603e-03 5.00000000e-01 8.33333333e-02]
 [1.79000000e+01 5.47945205e-03 6.66666667e-01 8.33333333e-02]
 [1.88000000e+01 8.21917808e-03 8.33333333e-01 8.33333333e-02]
 [1.46000000e+01 1.09589041e-02 1.00000000e+00 8.33333333e-02]
 [1.58000000e+01 1.36986301e-02 0.00000000e+00 8.33333333e-02]],  y[0]: 15.8
x[1]: [[1.79000000e+01 5.47945205e-03 6.66666667e-01 8.33333333e-02]
 [1.88000000e+01 8.21917808e-03 8.33333333e-01 8.33333333e-02]
 [1.46000000e+01 1.09589041e-02 1.00000000e+00 8.33333333e-02]
 [1.58000000e+01 1.36986301e-02 0.00000000e+00 8.33333333e-02]
 [1.58000000e+01 1.64383562e-02 1.66666667e-01 8.33333333e-02]],  y[1]: 17.4
x[2]: [[1.88000000e+01 8.21917808e-03 8.33333333e-01 8.33333333e-02]
 [1.46000000e+01 1.09589041e-02 1.00000000e+00 8.33333333e-02]
 [1.58000000e+01 1.36986301e-02 0.00000000e+00 8.33333333e-02]
 [1.58000000e+01 1.64383562e-02 1.66666667e-01 8.33333333e-02]
 [1.58000000e+01 1.91780822e-02 3.33333333e-01 8.33333333e-02]],  y[2]: 

In [106]:
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.regularizers import L1L2
from tensorflow.keras.layers import LSTM, Bidirectional

inputs = keras.layers.Input(shape=(5, 4))

# First layer - Bidirectional LSTM with regularization
x = Bidirectional(LSTM(128, return_sequences=True, kernel_regularizer=L1L2(l1=1e-5, l2=1e-4)))(inputs)
x = keras.layers.Dropout(0.3)(x)
x = keras.layers.LayerNormalization()(x)

# Second layer - Bidirectional LSTM
x = Bidirectional(LSTM(256, return_sequences=True, kernel_regularizer=L1L2(l1=1e-5, l2=1e-4)))(x)
x = keras.layers.Dropout(0.3)(x)
x = keras.layers.LayerNormalization()(x)

# Third layer - LSTM
x = LSTM(256, return_sequences=False, kernel_regularizer=L1L2(l1=1e-5, l2=1e-4))(x)
x = keras.layers.Dropout(0.3)(x)

# Dense layers before output
x = keras.layers.Dense(128, activation='relu')(x)
x = keras.layers.Dense(64, activation='relu')(x)

# Output layer
outputs = keras.layers.Dense(1)(x)

# Construcción y compilación del modelo
model = keras.Model(inputs=inputs, outputs=outputs)
model.compile(optimizer='adam', loss='mean_squared_error')
model.summary()

Model: "functional_27"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_13 (InputLayer)     │ (None, 5, 4)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional_15                │ (None, 5, 256)         │       136,192 │
│ (Bidirectional)                 │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_23 (Dropout)            │ (None, 5, 256)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ layer_normalization_14          │ (None, 5, 256)         │           512 │
│ (LayerNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional_16                │ (None, 5, 512)         │     1,050,624 │
│ (Bidirectional)                 │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_24 (Dropout)            │ (None, 5, 512)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ layer_normalization_15          │ (None, 5, 512)         │         1,024 │
│ (LayerNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_25 (LSTM)                  │ (None, 256)            │       787,456 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_25 (Dropout)            │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_28 (Dense)                │ (None, 128)            │        32,896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_29 (Dense)                │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_30 (Dense)                │ (None, 1)              │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,017,025 (7.69 MB)

 Trainable params: 2,017,025 (7.69 MB)

 Non-trainable params: 0 (0.00 B)

In [107]:
es_callback = keras.callbacks.EarlyStopping(
    monitor="val_loss", min_delta=0, patience=10)

history = model.fit(
    X_train, y_train,
    epochs=200,
    validation_split=0.2, shuffle=True, batch_size = 64, callbacks=[es_callback]
)

Epoch 1/200
38/38 ━━━━━━━━━━━━━━━━━━━━ 16s 127ms/step - loss: 47.6111 - val_loss: 19.5566
Epoch 2/200
38/38 ━━━━━━━━━━━━━━━━━━━━ 4s 97ms/step - loss: 15.8605 - val_loss: 9.1060
Epoch 3/200
38/38 ━━━━━━━━━━━━━━━━━━━━ 4s 98ms/step - loss: 10.4735 - val_loss: 9.5430
Epoch 4/200
38/38 ━━━━━━━━━━━━━━━━━━━━ 4s 98ms/step - loss: 10.9162 - val_loss: 9.2550
Epoch 5/200
38/38 ━━━━━━━━━━━━━━━━━━━━ 4s 95ms/step - loss: 10.2181 - val_loss: 9.4593
Epoch 6/200
38/38 ━━━━━━━━━━━━━━━━━━━━ 4s 97ms/step - loss: 10.2771 - val_loss: 9.3648
Epoch 7/200
38/38 ━━━━━━━━━━━━━━━━━━━━ 4s 97ms/step - loss: 10.2485 - val_loss: 9.3177
Epoch 8/200
38/38 ━━━━━━━━━━━━━━━━━━━━ 4s 100ms/step - loss: 10.4227 - val_loss: 9.2458
Epoch 9/200
38/38 ━━━━━━━━━━━━━━━━━━━━ 4s 96ms/step - loss: 10.2367 - val_loss: 9.2180
Epoch 10/200
38/38 ━━━━━━━━━━━━━━━━━━━━ 4s 96ms/step - loss: 10.0708 - val_loss: 8.9809
Epoch 11/200
38/38 ━━━━━━━━━━━━━━━━━━━━ 4s 97ms/step - loss: 9.8689 - val_loss: 9.3108
Epoch 12/200
38/38 ━━━━━━━━━━━━━━━━━━━

In [108]:
results = model.evaluate(X_test, y_test, verbose=1)
print('Test Loss: {}'.format(results))

21/21 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - loss: 6.0531
Test Loss: 6.513253211975098


<a name='3.4'></a>
## Cuestión 4: ¿En cuáles de estas aplicaciones se usaría un arquitectura 'many-to-one'?

**a)** Clasificación de sentimiento en textos

**b)** Verificación de voz para iniciar el ordenador.

**c)** Generación de música.

**d)** Un clasificador que clasifique piezas de música según su autor.


En una configuración "many-to-one", la red recibe una secuencia de entradas y produce una única salida al final de la secuencia. Este tipo de arquitectura es adecuada para tareas donde el contexto de la secuencia completa es necesario para producir una respuesta, como es el caso en:

**a) Clasificación de sentimiento en textos:** Se analiza una secuencia de palabras (la entrada) para determinar si el sentimiento general del texto es positivo, negativo o neutro (una única salida). Por tanto, es una aplicación "many-to-one".

**b) Verificación de voz para iniciar el ordenador:**  es "many-to-one" si se centra en clasificar la secuencia de audio como perteneciente a un usuario autorizado o no.

**d) Un clasificador que clasifique piezas de música según su autor:** Aquí, la entrada es una secuencia de notas o sonidos, y la salida es la clasificación del autor de la pieza, que es una única salida basada en la secuencia completa. Por lo tanto, es una aplicación "many-to-one".

<a name='3.5'></a>
## Cuestión 5: ¿Qué ventajas aporta el uso de word embeddings?

**a)** Permiten reducir la dimensión de entrada respecto al one-hot encoding.

**b)** Permiten descubrir la similaridad entre palabras de manera más intuitiva que con one-hot encoding.

**c)** Son una manera de realizar transfer learning en nlp.

**d)** Permiten visualizar las relaciones entre palabras con métodos de reducción de dimensioones como el PCA.


Las respuestas correctas serían todas las opciones:

**a) Permiten reducir la dimensión de entrada respecto al one-hot encoding.**  
Los word embeddings representan palabras en un espacio de características continuo y de menor dimensión en comparación con el one-hot encoding, donde la dimensión del vector es igual al tamaño del vocabulario. Esto reduce drásticamente la dimensión de los datos de entrada sin perder información sobre la palabra.

**b) Permiten descubrir la similaridad entre palabras de manera más intuitiva que con one-hot encoding.**  
En el espacio vectorial de los embeddings, palabras con significados similares o que se utilizan en contextos similares tienden a estar más cerca unas de otras, lo que permite capturar y utilizar la similaridad semántica de manera efectiva. En contraste, el one-hot encoding trata todas las palabras como igualmente distintas entre sí sin ninguna noción de similaridad.

**c) Son una manera de realizar transfer learning en NLP.**  
Los word embeddings pueden entrenarse en grandes conjuntos de datos y luego utilizarse en modelos más pequeños o tareas específicas, transfiriendo así el conocimiento aprendido sobre el lenguaje de un dominio a otro. Esto es especialmente útil en situaciones donde los datos de entrenamiento son limitados.

**d) Permiten visualizar las relaciones entre palabras con métodos de reducción de dimensiones como el PCA.**  
Dado que los word embeddings representan palabras en un espacio de características de alta dimensión, técnicas de reducción de dimensiones como el Análisis de Componentes Principales (PCA) o t-SNE pueden utilizarse para proyectar estos espacios a dos o tres dimensiones. Esto hace posible visualizar la relación entre palabras, mostrando cómo ciertas palabras están agrupadas o relacionadas entre sí en el espacio reducido.
